In [1]:
import pyspark
from pyspark.sql.functions import rand
import datetime

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType,FloatType, DoubleType

from pyspark import SparkContext, HiveContext
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import *
import argparse
import datetime
from pyspark.sql import SparkSession
from datetime import datetime
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 02:14:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
"""## Global Variables"""

MoneyMule = 50.00

code_not_to_use = [1, 18, 111, 30000, 31813, 32190, 33205, 33206, 37505, 99999 ]

high_risk_categories = ['VH', 'HI', 'PE']

so_scr = ['SO', 'SCR','']

"""## Criteria"""

A_Criteria = 499.99
B_Criteria = 499.99
C_Criteria = 4999.99
D_Criteria_Lower = 2499.99
D_Criteria_Upper = 4999.99
E_Criteria= 4999.99
F_Criteria = 499.99
G_Criteria = 2499.99

Y_Criteria = 500.00
Z_Criteria = 5999.99

Minor = 16
MuleLower = 17
MuleUpper = 25

user_id =1234

In [3]:
"""## AWS S3 File Path"""

trans_path = "/Users/Marium_Faheem/Library/CloudStorage/OneDrive-McKinsey&Company/Desktop/learning/maham-df/new/ScionTransactionList.csv"
member_path = "/Users/Marium_Faheem/Library/CloudStorage/OneDrive-McKinsey&Company/Desktop/learning/maham-df/new/ScionMemberList.csv"

member_df = spark.read.option("header", True).csv(member_path)

trans_df = spark.read.option("header", True).csv(trans_path)


In [4]:
"""## Data Pre-Processing"""

trans_df = trans_df.withColumnRenamed('Trans In Amount ', 'TransInAmount')
trans_df = trans_df.withColumnRenamed('Trans Out Amount', 'TransOutAmount')


pattern = r"\,"
trans_df = trans_df.withColumn(
    "TransInAmount",
    regexp_replace(trans_df["TransInAmount"], pattern, "")  # Replace comma with empty string
)

trans_df = trans_df.withColumn(
    "TransOutAmount",
    regexp_replace(trans_df["TransOutAmount"], pattern, "")  # Replace comma with empty string
)


"""# Calculate Columns from `Transaction File`"""

trans_df = trans_df.withColumn("TransactionTotal", col("TransInAmount") - col("TransOutAmount"))

trans_df = trans_df.withColumn(
    "TransactionPositiveTotal",
    when(col("TransactionTotal").isNull(), "").otherwise(abs(col("TransactionTotal")))
)

trans_df = trans_df.withColumn(
    "TransForMoneyMuleAndCombined",
    when(col("TransactionPositiveTotal") > MoneyMule, col("TransactionPositiveTotal")).otherwise(0)
)

trans_df = trans_df.withColumn(
    "Trans Incl",
    when(col("TransactionPositiveTotal") > 0, "Yes").otherwise("No")
)

trans_df = trans_df.withColumn(
    "Account_Not_In_Use",
    when(col("Account Number").isin(code_not_to_use), "No").otherwise("Yes")
)

#current = s0 or
trans_df = trans_df.withColumn(
    "SO_SCR",
    when(col("Transaction Type").isin(so_scr), col('Transaction Type')).otherwise("")
)

In [5]:
trans_df.printSchema()

root
 |-- Account Number: string (nullable = true)
 |-- Financial Month: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Transaction No: string (nullable = true)
 |-- Transaction Type: string (nullable = true)
 |-- Trans Type Category: string (nullable = true)
 |-- Transaction Note: string (nullable = true)
 |-- TransInAmount: string (nullable = true)
 |-- TransOutAmount: string (nullable = true)
 |-- TransactionTotal: double (nullable = true)
 |-- TransactionPositiveTotal: string (nullable = true)
 |-- TransForMoneyMuleAndCombined: string (nullable = true)
 |-- Trans Incl: string (nullable = false)
 |-- Account_Not_In_Use: string (nullable = false)
 |-- SO_SCR: string (nullable = true)



In [5]:
trans_df.filter(col("Transaction No")==46996114).show()

+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+
|            68|         23-Dec|07-Dec-23|      46996114|              SO|                EFT|   Not Available|            0|           -83|            83.0|                    83.0|                        83.0|       Yes|               Yes|    SO|
+---

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lead, lag


# Define the window specification
windowSpec = Window.orderBy("Account Number")

# Add previous, current, and next values
df_with_prev_next = trans_df.withColumn("previous_value", lag("SO_SCR").over(windowSpec)) \
    .withColumn("next_value", lead("SO_SCR").over(windowSpec))


# previous value = SCR and current value = s0 and next value = s0
df_with_prev_next =df_with_prev_next.withColumn(
    "No1",
    when((col("previous_value") == "SCR") & (col('SO_SCR') == "SO") & ( col('next_value') =="SO"), "Y").otherwise(" ")
)


df_with_prev_next.filter(col("Transaction No")==46996114).show()

24/03/16 02:14:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|previous_value|next_value|No1|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+
|            68|         23-Dec|07-Dec-23|      46996114|              SO|                EFT|   Not Available|            0|           -83|            83.0|      

In [7]:
#No2 previous n1 =’Y’ and  SO_SCR = s0. then Y else “”
df_with_prev_next = df_with_prev_next.withColumn(
    "No2",
    when(
        (lag("No1", 1).over(windowSpec) == "Y") & (col("SO_SCR") == "SO"),
        "Y"
    ).otherwise("")
)
df_with_prev_next.filter(col("Transaction No")==47012712).show()

24/03/16 02:14:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:14:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 0

+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+---+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|previous_value|next_value|No1|No2|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+---+
|            68|         23-Dec|14-Dec-23|      47012712|              SO|                EFT|   Not Available|           83|             0|           

In [8]:
#No 3 current No1 = Y and next No2 = Y and  current_total transaction = next_total trasacation then E
df_with_prev_next = df_with_prev_next.withColumn(
    "No3",
    when(
        (col("No1") == "Y") &
        (lead("No2", 1).over(windowSpec) == "Y") &
        (col("TransactionTotal") == lead("TransactionTotal", 1).over(windowSpec)),
        "E"
    ).otherwise("")
)

df_with_prev_next.filter(col("Transaction No")==47012711).show()

24/03/16 02:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 0

+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+---+---+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|previous_value|next_value|No1|No2|No3|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+---+---+
|            68|         23-Dec|14-Dec-23|      47012711|              SO|                EFT|   Not Available|            0|           -83

In [10]:
#No 4  - if previous no2 == E then E
df_with_prev_next = df_with_prev_next.withColumn(
    "No4",
    when(
        (lag("No3", 1).over(windowSpec) == "E"),
        "E"
    ).otherwise("")
)

#
df_with_prev_next.filter(col("Transaction No")==46996115).show()

24/03/16 02:17:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:17:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:17:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:17:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:17:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 02:17:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 0

+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+---+---+---+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|previous_value|next_value|No1|No2|No3|No4|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+----------+---+---+---+---+
|            68|         23-Dec|07-Dec-23|      46996115|              SO|                EFT|   Not Available|           83|  

In [13]:


"""# Joining the table"""

member_trans_df = trans_df.join(member_df,df_with_prev_next['Account Number']==df_with_prev_next['Account Number'], how="inner")

member_trans_df.show()

member_trans_df.filter(member_trans_df['Risk Code'] == "HI").show()

"""# Calculate Columns from `Member_trans_df`"""

member_trans_df = member_trans_df.withColumn(
    "HighRisk",
    when(col("Risk Code").isin(high_risk_categories), "Yes").otherwise("No")
)

# Account_Not_In_Use ki jaGw code_in_use ayeGw
member_trans_df = member_trans_df.withColumn("GreaterThan",
                                             when((col("Account_Not_In_Use")=='Yes') & (col("Trans Incl")=='Yes') & ((col("TransInAmount") >= D_Criteria_Lower) & (col("TransInAmount")<= D_Criteria_Upper)), "Yes")
                                             .otherwise("No"))

"""# Display the `member_trans_df` DataFrame"""

member_trans_df.show()

"""### Verification of the Column Created"""

member_trans_df.filter(col('GreaterThan') == "Yes").show(5)

member_trans_df.filter(col('HighRisk') == "Yes").show(5)

code no tot use = yes
# trans incl yes
# high risk yes
# trans postive total > A_Crtiera

"""# Creteria Creation"""

member_trans_df = member_trans_df.withColumn("A",
                                             when((col("Account_Not_In_Use")=="Yes") # Account_Not_In_Use ki jaga code_in_use ayeGw column
                                                  & (col("HighRisk")=="Yes")
                                                  & (col("TransactionPositiveTotal") > A_Criteria ), "A")
                                             .otherwise(None))

counts_of_A = member_trans_df.filter(member_trans_df['A'] == 'A').count()
print("Count of records where column 'A' equals 'A':", counts_of_A)
print("==========================================================")
#
# c_s3_bucket_path = "s3://scion-platform-bucket/source/demo/scion_criteria.csv"




s3_base_path = "s3://alm-platform-bucket/data_products/"
date_path = datetime.now().strftime('%Y/%m/%d/')
output_file_name = "scion_criteria.csv"
# Full path including the date
c_s3_bucket_path = s3_base_path +user_id +date_path + output_file_name
member_trans_df .write.mode("overwrite").option("header", "true").csv(c_s3_bucket_path)
print(f"Data saved to {c_s3_bucket_path}")


+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+--------------+------------+------------------+--------------------+---+-----------+--------------+---------+-----------+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|Account Number|Account Type| Employment Status| Employment Position|Age|Nationality|Account Status|Risk Code|Date Joined|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+------------

24/03/16 02:22:12 WARN Column: Constructing trivially true equals predicate, 'Account Number#52 = Account Number#52'. Perhaps you need to use aliases.
